In [1]:
from utils import *
from opt import *
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
from sklearn.covariance import graphical_lasso

# Define a function to handle parallel computation for each combination of mu1 and eta
def compute_estimate(mu1, eta, model):
    print('Mu: ', mu1, ' Eta: ', eta)
    if model == 'GL':
        Theta_hat = FairGLASSO_fista(C_hat, mu1, eta, mu2, Z, bias_type,
                                    epsilon=0.1, iters=1000, prec_type=None)
    elif model == 'GSR':
        Theta_hat = FairGSR_fista(C_hat, mu1, eta, mu2, Z, bias_type, iters=1000,
                     prec_type='non-negative', tol=1e-3, EARLY_STOP=False, RETURN_ITERS=False)
    elif model == 'GLASSO':
         _, Theta_hat = graphical_lasso(C_hat, alpha=mu1, max_iter=1000, tol=1e-3) 
    else:
        print('Unknown method')

    A_hat, A_hat_bin = extract_estimation(Theta_hat, A_true_bin)
    est_err = compute_frob_err(A_hat, A_true)
    est_f1 = compute_f1_score(A_hat_bin, A_true_bin)
    return est_err, est_f1

In [2]:
dnames = ['schoolCs4'] 

mus1 = np.logspace(-2,1,4) #sparsity 
etas = np.logspace(-3,1,10) #step size
mu2 = 0
nmus = len(mus1)
netas = len(etas)
bias_type = 'dp'
epsilon = 0.1
est_error = np.zeros((nmus, netas))
est_fsc = np.zeros((nmus, netas))

model = 'GSR'



for k, dn in enumerate(dnames):
    A_true, A_true_bin, C_est, C_est_norm, Z, z = load_datasets(dn)
    N = A_true.shape[0]
    C_hat = C_est_norm
    A_hat = np.zeros((N, N))
    A_hat_bin = np.zeros((N, N))

    # Define the number of cores to use for parallel processing
    num_cores = multiprocessing.cpu_count()
    # Parallel computation
    results = Parallel(n_jobs=num_cores)(
        delayed(compute_estimate)(mu1, eta, model)
        for mu1 in mus1
        for eta in etas
    )

    # Assign the results to the arrays
    for idx, (mu1_idx, eta_idx) in enumerate([(i, j) for i in range(nmus) for j in range(netas)]):
        est_error[mu1_idx, eta_idx], est_fsc[mu1_idx, eta_idx] = results[idx]


    show_params(mus1,etas,est_error,est_fsc,dn)
    p_grp = Z.sum(axis=1)
    print(p_grp)

schoolCs4 : Min. error at mu1 = 7 and eta = 0.006 Min. error value: 0.4404090977834826
schoolCs4 : Max. F1 score at mu1 = 7 and eta = 0.006 Max. F1 score value: 0.1961398017736046

[41 85]
